# Лабораторная работа №2: Исследование логистической и линейной регрессий

**Цель:** Изучить применение логистической регрессии для классификации и линейной регрессии для регрессионных задач.

## Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
mean_absolute_error, mean_squared_error, r2_score
)

import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка датасета для классификации (UCI Credit Card)

In [2]:
df = pd.read_csv("UCI_Credit_Card.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


## Анализ структуры данных

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


## Анализ распределения целевой переменной

In [4]:
df['default.payment.next.month'].value_counts(normalize=True)

default.payment.next.month
0    0.7788
1    0.2212
Name: proportion, dtype: float64

## Подготовка данных для моделирования

In [5]:
X = df.drop(columns=['default.payment.next.month', 'ID'])
y = df['default.payment.next.month']

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42, stratify=y
)

## Создание пайплайна для логистической регрессии

In [6]:
logreg_pipeline = Pipeline([
('scaler', StandardScaler()),
('model', LogisticRegression(max_iter=1000))
])

logreg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', LogisticRegression(max_iter=1000))])

## Получение предсказаний

In [7]:
y_pred = logreg_pipeline.predict(X_test)
y_proba = logreg_pipeline.predict_proba(X_test)[:, 1]

In [8]:
metrics_classification = {
'Accuracy': accuracy_score(y_test, y_pred),
'Precision': precision_score(y_test, y_pred),
'Recall': recall_score(y_test, y_pred),
'F1-score': f1_score(y_test, y_pred),
'ROC-AUC': roc_auc_score(y_test, y_proba)
}

pd.Series(metrics_classification)

Accuracy     0.807667
Precision    0.686825
Recall       0.239638
F1-score     0.355307
ROC-AUC      0.707636
dtype: float64

## Оценка качества базовой модели логистической регрессии
**Результаты:** 
- Accuracy: 0.8077 - высокая общая точность
- Precision: 0.6868 - из предсказанных дефолтов 68.7% действительно дефолт
- Recall: 0.2396 - только 24% реальных дефолтов найдено
- F1: 0.3553 - низкий из-за дисбаланса
- ROC-AUC: 0.7076 - среднее качество

**Вывод:** Модель имеет высокую точность, но низкий recall - типичная проблема при несбалансированных данных.

## Загрузка датасета для регрессии (Air Quality)

In [9]:
df_air = pd.read_csv("AirQuality.csv", sep=';', decimal=',')
df_air.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


## Предобработка данных о качестве воздуха
1. Удаляем пустые столбцы
2. Заменяем -200 (пропуски) на NaN
3. Удаляем строки с пропусками
4. Получаем чистый датасет из 15 колонок

In [10]:
df_air = df_air.dropna(axis=1, how='all')

df_air = df_air.replace(-200, np.nan)

df_air = df_air.dropna()

## Подготовка данных для линейной регрессии

In [11]:
target = 'CO(GT)'

X = df_air.drop(columns=[target, 'Date', 'Time'])
y = df_air[target]

## Разделение на train/test для регрессии

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

## Создание пайплайна для линейной регрессии

In [13]:
linreg_pipeline = Pipeline([
('scaler', StandardScaler()),
('model', LinearRegression())
])

linreg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

## Обучение линейной регрессии

In [14]:
y_pred = linreg_pipeline.predict(X_test)

## Получение предсказаний для регрессии


In [15]:
metrics_regression = {
'MAE': mean_absolute_error(y_test, y_pred),
'RMSE': mean_squared_error(y_test, y_pred, squared=False),
'R2': r2_score(y_test, y_pred)
}

pd.Series(metrics_regression)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MAE     0.177941
RMSE    0.240677
R2      0.972173
dtype: float64

## Оценка качества базовой линейной регрессии
**Результаты:**
- MAE: 0.1779 - средняя абсолютная ошибка
- RMSE: 0.2407 - корень из среднеквадратичной ошибки
- R²: 0.9722 - очень высокий коэффициент детерминации

**Вывод:** Модель отлично объясняет дисперсию данных (R² = 0.97).

## Улучшение логистической регрессии: учет дисбаланса классов
Используем class_weight='balanced' для автоматической балансировки весов классов.

In [16]:
X = df.drop(columns=['default.payment.next.month', 'ID'])
y = df['default.payment.next.month']

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42, stratify=y
)

In [17]:
logreg_balanced = Pipeline([
('scaler', StandardScaler()),
('model', LogisticRegression(
max_iter=1000,
class_weight='balanced'
))
])

logreg_balanced.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

## Обучение сбалансированной модели
Обучаем логистическую регрессию с учетом дисбаланса.

In [18]:
y_pred = logreg_balanced.predict(X_test)
y_proba = logreg_balanced.predict_proba(X_test)[:, 1]

In [19]:
pd.Series({
'Accuracy': accuracy_score(y_test, y_pred),
'Precision': precision_score(y_test, y_pred),
'Recall': recall_score(y_test, y_pred),
'F1': f1_score(y_test, y_pred),
'ROC-AUC': roc_auc_score(y_test, y_proba)
})

Accuracy     0.679667
Precision    0.367247
Recall       0.620196
F1           0.461323
ROC-AUC      0.708115
dtype: float64

## Результаты сбалансированной модели
**Сравнение с базовой:**
- Accuracy: 0.6797 → 0.8077 (снизилась)
- Precision: 0.3672 → 0.6868 (снизилась)
- Recall: 0.6202 → 0.2396 (увеличился в 2.6 раза)
- F1: 0.4613 → 0.3553 (улучшился)
- ROC-AUC: 0.7081 → 0.7076 (незначительное изменение)

**Вывод:** Балансировка значительно улучшила recall (способность находить дефолты) ценой снижения precision.

## Подбор гиперпараметров через GridSearchCV
Ищем оптимальное значение параметра регуляризации C для сбалансированной модели.

## Обучение GridSearchCV
Ищем лучшие параметры с кросс-валидацией (5 фолдов) по метрике F1.

## Предсказания лучшей модели
Используем модель с оптимальными параметрами.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
'model__C': [0.01, 0.1, 1, 10]
}

pipeline = Pipeline([
('scaler', StandardScaler()),
('model', LogisticRegression(
max_iter=1000,
class_weight='balanced'
))
])

grid = GridSearchCV(
pipeline,
param_grid,
cv=5,
scoring='f1',
n_jobs=-1
)

grid.fit(X_train, y_train)

grid.best_params_

{'model__C': 0.01}

## Лучшие параметры
Наилучшее значение C = 0.01.

## Предсказания лучшей модели
Используем модель с оптимальными параметрами.

In [21]:
best_model = grid.best_estimator_

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

In [22]:
metrics_classification_improved = pd.Series({
'Accuracy': accuracy_score(y_test, y_pred),
'Precision': precision_score(y_test, y_pred),
'Recall': recall_score(y_test, y_pred),
'F1': f1_score(y_test, y_pred),
'ROC-AUC': roc_auc_score(y_test, y_proba)
})

metrics_classification_improved

Accuracy     0.679500
Precision    0.367083
Recall       0.620196
F1           0.461194
ROC-AUC      0.707595
dtype: float64

## Оценка улучшенной модели
**Итоговые результаты:** Почти идентичны простой сбалансированной модели.
**Вывод:** Для данной задачи простая балансировка дает основной выигрыш, тонкая настройка C незначительно влияет.

## Улучшение линейной регрессии: Ridge регрессия
Используем Ridge регрессию с L2 регуляризацией для борьбы с переобучением.

In [23]:
target = 'CO(GT)'

X = df_air.drop(columns=[target, 'Date', 'Time'])
y = df_air[target]

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

## Подбор alpha для Ridge регрессии
Ищем оптимальный параметр регуляризации alpha через GridSearchCV.

In [24]:
from sklearn.linear_model import Ridge

param_grid = {
'model__alpha': [0.1, 1, 10, 50]
}

ridge_pipeline = Pipeline([
('scaler', StandardScaler()),
('model', Ridge())
])

grid_ridge = GridSearchCV(
ridge_pipeline,
param_grid,
cv=5,
scoring='r2',
n_jobs=-1
)

grid_ridge.fit(X_train, y_train)

grid_ridge.best_params_

{'model__alpha': 0.1}

## Лучший параметр alpha = 0.1
Слабая регуляризация, что указывает на отсутствие сильного переобучения в базовой модели.

In [25]:
best_ridge = grid_ridge.best_estimator_

y_pred = best_ridge.predict(X_test)

In [26]:
metrics_regression_improved = pd.Series({
'MAE': mean_absolute_error(y_test, y_pred),
'RMSE': mean_squared_error(y_test, y_pred, squared=False),
'R2': r2_score(y_test, y_pred)
})

metrics_regression_improved

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MAE     0.178048
RMSE    0.240624
R2      0.972186
dtype: float64

## Оценка улучшенной Ridge регрессии
**Результаты:** Почти идентичны базовой линейной регрессии.
**Вывод:** Базовая линейная регрессия уже работает отлично, регуляризация не дает значимого улучшения.

# Имплементация алгоритма машинного обучения

In [ ]:
class LogisticRegressionGD:
    def __init__(self, lr=0.01, n_iters=1000, l2=0.0, class_weight=None):
        self.lr = lr
        self.n_iters = n_iters
        self.l2 = l2
        self.class_weight = class_weight

    def _sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        if self.class_weight == 'balanced':
            w0 = len(y) / (2 * np.sum(y == 0))
            w1 = len(y) / (2 * np.sum(y == 1))
            sample_weights = np.where(y == 1, w1, w0)
        else:
            sample_weights = np.ones_like(y)

        for _ in range(self.n_iters):
            linear = np.dot(X, self.w) + self.b
            y_pred = self._sigmoid(linear)
            error = y_pred - y

            dw = (1 / n_samples) * np.dot(X.T, error * sample_weights) + self.l2 * self.w
            db = (1 / n_samples) * np.sum(error * sample_weights)

            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict_proba(self, X):
        return self._sigmoid(np.dot(X, self.w) + self.b)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)


## Cтандартизация

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

logreg_custom = LogisticRegressionGD(lr=0.05, n_iters=1500)
logreg_custom.fit(X_train, y_train)

## Для регрессионного датасета нужно преобразовать задачу в бинарную классификацию.

## Преобразование задачи регрессии в классификацию
Создаем бинарную целевую переменную: выше/ниже медианы концентрации CO.

In [ ]:
y_binary = (y > y.median()).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_binary, test_size=0.2, random_state=42
)

logreg_custom = LogisticRegressionGD(lr=0.01, n_iters=1500)
logreg_custom.fit(X_train, y_train)

y_pred = logreg_custom.predict(X_test)
y_proba = logreg_custom.predict_proba(X_test)

metrics_custom_cls = pd.Series({
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1': f1_score(y_test, y_pred),
    'ROC-AUC': roc_auc_score(y_test, y_proba)
})

In [31]:
logreg_custom_improved = LogisticRegressionGD(
lr=0.1,
n_iters=2500,
l2=0.01,
class_weight='balanced'
)

logreg_custom_improved.fit(X_train, y_train)

In [32]:
y_pred = logreg_custom_improved.predict(X_test)
y_proba = logreg_custom_improved.predict_proba(X_test)

metrics_custom_cls_improved = pd.Series({
'Accuracy': accuracy_score(y_test, y_pred),
'Precision': precision_score(y_test, y_pred),
'Recall': recall_score(y_test, y_pred),
'F1': f1_score(y_test, y_pred),
'ROC-AUC': roc_auc_score(y_test, y_proba)
})
metrics_custom_cls_improved

Accuracy     0.969880
Precision    0.973684
Recall       0.961039
F1           0.967320
ROC-AUC      0.994747
dtype: float64

## Обучение и оценка собственной модели
**Результаты:** Отличные метрики (Accuracy: 0.97, F1: 0.967)
**Вывод:** Собственная реализация работает хорошо на преобразованной задаче.

## Обучение собственной линейной регрессии
Обучаем на исходной регрессионной задаче (прогноз концентрации CO).

In [40]:
class LinearRegressionGD:
    def __init__(self, lr=0.01, n_iters=3000, l2=0.0, adaptive_lr=True, early_stopping=True, patience=100):
        self.lr = lr
        self.n_iters = n_iters
        self.l2 = l2
        self.adaptive_lr = adaptive_lr
        self.early_stopping = early_stopping
        self.patience = patience
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0
        self.loss_history = []
        
        best_loss = float('inf')
        patience_counter = 0
        best_w = self.w.copy()
        best_b = self.b
        
        self.w = np.random.randn(n_features) * 0.01
        
        for i in range(self.n_iters):
            y_pred = np.dot(X, self.w) + self.b
            error = y_pred - y

            mse = np.mean(error ** 2) + self.l2 * np.sum(self.w ** 2) / n_samples
            self.loss_history.append(mse)
            
            dw = (2 / n_samples) * np.dot(X.T, error) + (2 * self.l2 / n_samples) * self.w
            db = (2 / n_samples) * np.sum(error)
            
            if self.adaptive_lr:
                current_lr = self.lr / (1 + 0.001 * i)
            else:
                current_lr = self.lr
            
            if not hasattr(self, 'momentum_w'):
                self.momentum_w = np.zeros_like(dw)
                self.momentum_b = 0
            
            beta = 0.9
            self.momentum_w = beta * self.momentum_w + (1 - beta) * dw
            self.momentum_b = beta * self.momentum_b + (1 - beta) * db
            
            self.w -= current_lr * self.momentum_w
            self.b -= current_lr * self.momentum_b
            
            if self.early_stopping:
                if mse < best_loss - 1e-6:
                    best_loss = mse
                    best_w = self.w.copy()
                    best_b = self.b
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= self.patience:
                    print(f"Early stopping at iteration {i}")
                    self.w = best_w
                    self.b = best_b
                    break
        
        if self.early_stopping and i == self.n_iters - 1:
            self.w = best_w
            self.b = best_b
    
    def predict(self, X):
        return np.dot(X, self.w) + self.b

In [41]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [42]:
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()

In [43]:
linreg_gd_improved = LinearRegressionGD(
    lr=0.05,         
    n_iters=5000,   
    l2=0.01,          
    adaptive_lr=True, 
    early_stopping=True, 
    patience=200     
)

In [44]:
linreg_gd_improved.fit(X_train, y_train_scaled)

In [45]:
y_pred_scaled = linreg_gd_improved.predict(X_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

metrics_improved = pd.Series({
    'MAE': mean_absolute_error(y_test, y_pred),
    'RMSE': mean_squared_error(y_test, y_pred, squared=False),
    'R2': r2_score(y_test, y_pred)
})

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [46]:
print(metrics_improved)

MAE     0.180876
RMSE    0.241909
R2      0.971888
dtype: float64


## Сравнение результатов моделей линейной регрессии

### Таблица сравнения метрик

| Модель | MAE | RMSE | R² | Примечания |
|--------|-----|------|----|------------|
| **Sklearn LinearRegression** | 0.1779 | 0.2407 | 0.9722 | Базовая модель из библиотеки |
| **Собственная реализация** | 0.1809 | 0.2419 | 0.9719 | С адаптивным LR, моментом и ранней остановкой |


Собственная реализация линейной регрессии с градиентным спуском была успешно улучшена путем добавления адаптивного learning rate, момента, ранней остановки и масштабирования данных. В результате модель достигла качества, практически идентичного реализации в scikit-learn: R² = 0.9719 против 0.9722, что составляет разницу всего 0.03%. Это демонстрирует корректность реализации и эффективность применённых оптимизаций.